## Урок 4.  Рекомендательные системы на основе контента

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
        
# from metrics import precision_at_k, recall_at_k
# from utils import prefilter_items

In [2]:
data = pd.read_csv('recsys_retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
# item_features = pd.read_csv('product.csv')
# item_features.columns = [col.lower() for col in item_features.columns]
# item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

# item_features.head(2)

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
# n_items_before = data_train['item_id'].nunique()

# data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=5000)

# n_items_after = data_train['item_id'].nunique()
# print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

In [7]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [10]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 6.47 s


-----

# Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [11]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        # your_code.
        self.data = data
        
        self.user_item_matrix = self.prepare_matrix()  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     

    def prepare_matrix(self, index: str='user_id', columns: str='item_id', values: str='quantity'):
        # your_code

        user_item_matrix = pd.pivot_table(self.data, 
                                          index=index,
                                          columns=columns, 
                                          values=values,
                                          aggfunc='count', 
                                          fill_value=0)

        self.user_item_matrix = user_item_matrix.astype(float) 
        return user_item_matrix
    
    
    def prepare_dicts(self, user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    

    def fit(self, user_item_matrix, n_factors=50, regularization=0.7, iterations=5, num_threads=2):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return model
    
    
    def get_similar_items_recommendation(self, item, N=5):
        
#     """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    
        # your_code
        rec_list_s = []
        recs = self.model.similar_items(self.itemid_to_id[item], N=N+1)
        for ii in range(1, len(recs)):
            rec_list_s.append(self.id_to_itemid[recs[ii][0]])

        return rec_list_s

    def get_similar_users(self, user, N=5):
#     """Находим похожих юзеров"""
        
        rec_list_u = []
        recs = self.model.similar_users(self.userid_to_id[user], N=N+1)
        for ii in range(1, len(recs)):
            rec_list_u.append(self.id_to_userid[recs[ii][0]])

        return rec_list_u

    
    def get_similar_users_recommendation(self, user, N=5):
#     """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        # your_code
    
        similars_list = self.get_similar_users(user, N=5)

        data_ = self.data[['user_id', 'item_id']].loc[self.data['user_id'].isin(similars_list)] 
        rec_list_items = data_['item_id'].value_counts()[:N].index.tolist()
        
        return rec_list_items
        
#         assert len(res) == N, 'Количество рекомендаций != {}'.format(N)


In [12]:
recommender = MainRecommender(data_train)

In [13]:
def get_items_top_n_for_users(data, N=5):
    assert ('quantity' or 'user_id' or 'item_id') in data.columns.tolist(),'quantity, user_id, item_id must be in data.columns' 
    
    popularity = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
    popularity.sort_values('quantity', ascending=False, inplace=True)

    popularity = popularity.groupby('user_id').head(N)

    popularity.sort_values(by=['user_id','quantity'], ascending=False, inplace=True)

    data_pop = popularity.groupby('user_id')['item_id'].unique().reset_index()
    data_pop.columns=['user_id', 'items_top_n']

    return data_pop

In [14]:
data_pop = get_items_top_n_for_users(data_train)
data_pop

,user_id,items_top_n
0,1,"[856942, 1082185, 995242, 940947, 9527290]"
1,2,"[1133018, 1106523, 8090521, 5569230, 916122]"
2,3,"[1092026, 1053690, 951590, 998206, 910032]"
3,4,"[883932, 6391541, 891423, 962229, 1052294]"
4,5,"[6904776, 6773079, 5995628, 1126899, 1123022]"
...,...,...
2494,2496,"[981760, 916122, 883404, 1056509, 5569230]"
2495,2497,"[860776, 995785, 1066685, 965719, 896938]"
2496,2498,"[1070820, 1126899, 1082185, 961554, 1053690]"
2497,2499,"[1070820, 826249, 5568378, 999971, 944317]"


In [15]:
# Проверим правильность группировки
data_train.loc[data_train['user_id'] == 1]['item_id'].value_counts()

856942      50
1082185     37
995242      32
940947      28
9527290     27
            ..
838738       1
12301502     1
9837544      1
9655175      1
1138467      1
Name: item_id, Length: 596, dtype: int64

In [16]:
# Передаем топовую item_id покупателя
recommender.get_similar_items_recommendation(856942)

[5566027, 960732, 1110031, 9655212, 865178]

In [17]:
# Передаем user_id и получаем рекомендации для этого пользователя
recommender.get_similar_users_recommendation(1)

[1082185, 854852, 8203451, 1077975, 854754]

----

Проверка, что все работает

In [18]:
from recsys_metrics import precision_at_k, recall_at_k
from recsys_utils import prefilter_items
from recsys_rec import MainRecommender

